In [31]:
import pandas as pd
from sklearn.metrics import classification_report

In [32]:
df = pd.read_csv("/Users/jennihawk/Documents/Data Science Projects/Project_NLP/Advanced_NLP_Project/Human Coherence Scoring/BERTopic_Scoring/bertopic_no_dupes_v2_coburn.csv")

df.head()

,Unnamed: 0,Document,Topic,Name,intruder,intruder_index,Coburn Annotator
0,0,I guess the government finally gave Trump an o...,0,0_subpoena_committee_subpoenaed_trump,0,NaN,NaN
1,1,Trump responds to Jan. 6 committee subpoena vo...,0,0_subpoena_committee_subpoenaed_trump,0,NaN,NaN
2,2,RT @PithyTruth: @JonathanTurley BEING A TRUMP ...,0,0_subpoena_committee_subpoenaed_trump,0,NaN,NaN
3,3,"Jan. 6 panel subpoenas Trump, shows startling ...",0,0_subpoena_committee_subpoenaed_trump,0,NaN,NaN
4,4,#HATH #January6thCommitteeHearings \n#Pennsylv...,1,1_trumpcoupattempt_trump_day_trumpisgoingtojail,1,4.0,1.0


In [33]:
# fill nans with 0 so chris_intruder and intruder look the same
df["Coburn Annotator"] = df["Coburn Annotator"].fillna(0)

#true data | ground truth
y_true = df["intruder"]

# model predictions - Human is also a model so this is our human prediction
y_pred = df["Coburn Annotator"]

In [34]:
df.shape

(60, 7)

### Model Scoring

In [37]:
print(
    classification_report(
        y_true, y_pred, target_names=["Does not matter", "Doc intrusion"]
    )
)

                 precision    recall  f1-score   support

Does not matter       0.90      0.90      0.90        48
  Doc intrusion       0.58      0.58      0.58        12

       accuracy                           0.83        60
      macro avg       0.74      0.74      0.74        60
   weighted avg       0.83      0.83      0.83        60



### Topic Scoring
- Using this function to create a new column that assigns a name to every group that ALSO includes the intruder in that group. The important point being that we need to get the intruder in the group name. 
- Grouping by every 5 rows because that's how data is structured. 
- Uses floor division to get an integer to use 
- IMPORTANT: If I change row number in research methodology - must update this code!

In [40]:
def add_majority_vote_column(df):
    # Helper function to determine the majority vote for a group
    def majority_vote(group):
        return group.mode()[0] if not group.mode().empty else None

    # Group rows by every 5 rows using floor division. Gets integer vs float.
    df['group'] = df.index // 5

    # Compute the majority vote for each group and map it to the original dataframe
    df['majority_vote'] = df['group'].map(df.groupby('group')['Name'].apply(majority_vote))

    # Drop the temporary 'group' column
    df.drop('group', axis=1, inplace=True)

    return df

#### Review what we're getting from function
- A column that identifies each grouping AND includes the intruder with that grouping. 

In [41]:
#add_majority_vote_column(df)

#### Create a new df to add the new column

In [42]:
#creates a new dataframe that adds the 'add_majority_vote_column'
new_df = add_majority_vote_column(df)

#### Create a filter to select the rows in topic 0

In [52]:
filt_0 = new_df.majority_vote.str.contains('0')
filt_1 = new_df.majority_vote.str.contains('1')
filt_2 = new_df.majority_vote.str.contains('2')
filt_3 = new_df.majority_vote.str.contains('3')

### Topic 0 Scoring
- Create another df and apply filter so we're isolating topic 0

In [48]:
xdf = new_df[filt_0]

In [49]:
# fill nans with 0 so chris_intruder and intruder look the same
xdf["Coburn Annotator"] = xdf["Coburn Annotator"].fillna(0)

#true data | ground truth
y_true = xdf["intruder"]

# model predictions - Human is also a model so this is our human prediction
y_pred = xdf["Coburn Annotator"]

/var/folders/j0/364009cj7xnbq46wqpdh5v9m0000gn/T/ipykernel_46589/389952767.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xdf["Coburn Annotator"] = xdf["Coburn Annotator"].fillna(0)


In [50]:
print(
    classification_report(
        y_true, y_pred, target_names=["Does not matter", "Doc intrusion"]
    )
)

                 precision    recall  f1-score   support

Does not matter       1.00      1.00      1.00        12
  Doc intrusion       1.00      1.00      1.00         3

       accuracy                           1.00        15
      macro avg       1.00      1.00      1.00        15
   weighted avg       1.00      1.00      1.00        15



#### Finding: topic 0 == 100% coherence

### Topic 1 Scoring

In [53]:
xdf1 = new_df[filt_1]

In [54]:
# fill nans with 0 so chris_intruder and intruder look the same
xdf1["Coburn Annotator"] = xdf1["Coburn Annotator"].fillna(0)

#true data | ground truth
y_true = xdf1["intruder"]

# model predictions - Human is also a model so this is our human prediction
y_pred = xdf1["Coburn Annotator"]

/var/folders/j0/364009cj7xnbq46wqpdh5v9m0000gn/T/ipykernel_46589/1515661031.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xdf1["Coburn Annotator"] = xdf1["Coburn Annotator"].fillna(0)


In [55]:
print(
    classification_report(
        y_true, y_pred, target_names=["Does not matter", "Doc intrusion"]
    )
)

                 precision    recall  f1-score   support

Does not matter       0.75      0.75      0.75        12
  Doc intrusion       0.00      0.00      0.00         3

       accuracy                           0.60        15
      macro avg       0.38      0.38      0.38        15
   weighted avg       0.60      0.60      0.60        15



### Topic 2 Scoring

In [56]:
xdf2 = new_df[filt_2]

In [57]:
# fill nans with 0 so chris_intruder and intruder look the same
xdf2["Coburn Annotator"] = xdf2["Coburn Annotator"].fillna(0)

#true data | ground truth
y_true = xdf2["intruder"]

# model predictions - Human is also a model so this is our human prediction
y_pred = xdf2["Coburn Annotator"]

/var/folders/j0/364009cj7xnbq46wqpdh5v9m0000gn/T/ipykernel_46589/182254506.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xdf2["Coburn Annotator"] = xdf2["Coburn Annotator"].fillna(0)


In [58]:
print(
    classification_report(
        y_true, y_pred, target_names=["Does not matter", "Doc intrusion"]
    )
)

                 precision    recall  f1-score   support

Does not matter       0.92      0.92      0.92        12
  Doc intrusion       0.67      0.67      0.67         3

       accuracy                           0.87        15
      macro avg       0.79      0.79      0.79        15
   weighted avg       0.87      0.87      0.87        15



### Topic 3 Scoring

In [59]:
xdf3 = new_df[filt_3]

In [60]:
# fill nans with 0 so chris_intruder and intruder look the same
xdf3["Coburn Annotator"] = xdf3["Coburn Annotator"].fillna(0)

#true data | ground truth
y_true = xdf3["intruder"]

# model predictions - Human is also a model so this is our human prediction
y_pred = xdf3["Coburn Annotator"]

/var/folders/j0/364009cj7xnbq46wqpdh5v9m0000gn/T/ipykernel_46589/3969133848.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xdf3["Coburn Annotator"] = xdf3["Coburn Annotator"].fillna(0)


In [61]:
print(
    classification_report(
        y_true, y_pred, target_names=["Does not matter", "Doc intrusion"]
    )
)

                 precision    recall  f1-score   support

Does not matter       0.92      0.92      0.92        12
  Doc intrusion       0.67      0.67      0.67         3

       accuracy                           0.87        15
      macro avg       0.79      0.79      0.79        15
   weighted avg       0.87      0.87      0.87        15



In [51]:
print(df['Name'].unique())

['0_subpoena_committee_subpoenaed_trump'
 '1_trumpcoupattempt_trump_day_trumpisgoingtojail'
 '2_and_forthepeople_peopleoverpolitics_standwithukraine'
 '3_rt_january6thcommitteehearings_watching_httpstcofcaejdpxxi']
